In [ ]:
import os
import sys
import shutil
import requests
import subprocess
import numpy as np
import pandas as pd
from pathlib import Path 
from tqdm.notebook import tqdm
    
from function.seqfilter import SeqFilter
from function.utilities import seq_to_fasta
from function.utilities import get_uniprot_rawdata

In [ ]:
#RaptorX pipeline
class HelixPropByRaptorX():
    """
    get RaptorX score,
    please download RaptorX from https://github.com/realbigws/Predict_Property,
    and compile Predict_Property as their instructions
    """
    def __init__(self, raptorx_path, output_path,sequence_profile=False):
        self.raptorx_path = Path(raptorx_path)
        self.output_path = Path(output_path)
        self.sequence_profile = sequence_profile
    
    def pipeline_get_seq_prop(self, fasta_path, od_index, helix_threshold, prop_score_filter_length):
        
        #run raptorx
        self.run_raptorx(fasta_path)
        
        #get helix score from raptorx output
        helix_score = self.parse_helix_score()
        
        #get helix count by threshold
        #add list func 
        if type(helix_threshold) is list:
            helix_counts_list = []
            for i in helix_threshold:
                helix_counts = self.get_helix_counts_perseq(od_index,helix_score,i, prop_score_filter_length)
                helix_counts_list.append(helix_counts)
            return helix_counts_list
        else:
            helix_counts = self.get_helix_counts_perseq(od_index,helix_score,helix_threshold, prop_score_filter_length)
            return helix_counts
    
    def run_raptorx(self, fasta_path):
        #run sequence profile
        if self.sequence_profile:
            pass
            #TODO
            
        #run raptorx
        raptorx_pp_runfile =  self.raptorx_path/ "Predict_Property.sh"
        self.output_path.mkdir(parents=True, exist_ok=True)
        self.output_path = self.output_path.absolute()
        
        cmd = "{} -i {} -o {}".format(str(raptorx_pp_runfile),str(fasta_path), str(self.output_path))
        try:
            ps = subprocess.check_call(cmd,
                                       shell=True,
                                       stdout=subprocess.PIPE,
                                       stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as e:
            ret_code = e.returncode
            print('An error occurred.  Error code:', e)
    
    def parse_helix_score(self):
        '''
        parse raptorx's output
        '''  
        parse_file_path = self.output_path/"tmp.ss3"
        df = pd.read_csv(parse_file_path,skiprows=[0,1],delim_whitespace=True,
                     names=['index','aa','indi','helix','beta','coil'])
        #clean old file
        shutil.rmtree(self.output_path)

        return df['helix'].tolist()

    def get_helix_counts_perseq(self, od_index, helix_score, helix_threshold, prop_score_filter_length):
        
        helix_counts = 0
        
        for i in od_index:
            start = i['start']
            end = i['end']

            frag_helix_score = helix_score[start:end]

            score_str = ''
            for i in frag_helix_score:
                if i > helix_threshold:
                    score_str = score_str + 'p'
                else:
                    score_str = score_str + ' '
            possible_propensity = list(filter(None, score_str.split(" ")))
            frag_helix_counts = len(list(filter(lambda item: len(item)>=prop_score_filter_length, possible_propensity)))
            helix_counts = frag_helix_counts + helix_counts
        
        return helix_counts

In [ ]:
#load human sequence and order/disroder information
human_df = get_uniprot_rawdata("./rawdata/human_uniprot.tab")
od_human_df = pd.read_pickle("./rawdata/VSL2_od_human_df.pkl")
def get_sequence(uniprot_id,return_sequence):
    if return_sequence == 'od_ident':
        return od_human_df[od_human_df['uniprot_id'] == uniprot_id]['od_ident'].tolist()[0]
    elif return_sequence == 'sequence':
        return human_df[human_df['uniprot_id'] == uniprot_id]['protein_sequence'].tolist()[0]

# Param

In [ ]:
#####CHANGE HERE#####

#RaptorX's output path for temporary saving output data, will be cleaned after each run
tmp_path = Path('/home/wenlin/tmp') 
fasta_path = tmp_path / 'tmp.fasta'
output_path = tmp_path / 'tmp_output'

#RaptorX Predict_Property program path, 
raptorx_path = "/home/wenlin/d/code/raptorx/Predict_Property"

#raptorx param
prop_score_filter_length = 10
helix_thresholds = [0.3,0.5,0.7,0.8,0.9]

#order/disorder length filter param
order_filter_length = 10
disorder_filter_length = 40

#####CHANGE HERE#####

# Run raptorx for all proteins

In [ ]:
seqfilter = SeqFilter()
hexliprop = HelixPropByRaptorX(raptorx_path,output_path)

raptorx_df = pd.DataFrame(human_df['uniprot_id'])

for index, row in tqdm(raptorx_df.iterrows(), total=raptorx_df.shape[0]):
    
    uniprot_id = row['uniprot_id']
    
    #get sequence
    sequence = get_sequence(uniprot_id,"sequence")

    #get disorder info
    #exception for no pondr disorder/order seq
    try:
        od_ident = get_sequence(uniprot_id,"od_ident")
    except Exception as e: 
        print(e)
        continue
        
    od_ident = seqfilter.length_filter_by_od_ident(od_ident,disorder_filter_length,order_filter_length) 
    od_index = seqfilter.get_od_index(od_ident)['disorder_region']

    #create fasta path
    seq_to_fasta(uniprot_id,sequence,fasta_path)
    
    #get score
    #exception for raptorx too long sequence 
    try:
        helix_props = hexliprop.pipeline_get_seq_prop(fasta_path, od_index, helix_thresholds, prop_score_filter_length)
        for threshold, prop in zip(helix_thresholds, helix_props):
            column_names='cond_{}'.format(threshold)
            raptorx_df.loc[raptorx_df['uniprot_id'] == uniprot_id, column_names] = prop
    except Exception as e:
        print("run raptorx failed: {}".format(uniprot_id))

In [ ]:
#save output to ./output
# df.to_pickle(".output/helix_prop_raptorx_longerthan_{}.pkl".format(prop_score_filter_length))